In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("oorschnai/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

2023-01-24 20:23:07.155582: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 20:23:07.371679: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 20:23:07.375434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 20:23:07.375446: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
import json

f = open('documents.json')
data = json.load(f)

f.close()

In [3]:
#добавлены координаты начала и конца, чтобы затем восстанавливать слова (тк есть обрывки)
# PER, ORG, MISC, LOC
def whatEntityIsThis(ent, entities):
    if ent == "I-PER" or ent == "B-PER":
        entities[0] += 1
        
    elif ent == "I-ORG" or ent == "B-ORG":
        entities[1] += 1
        
    elif ent == "I-LOC" or ent == "B-LOC":
        entities[2] += 1
        
    elif ent == "I-MISC" or ent == "B-MISC":
        entities[3] += 1


        
def choose_one_entity(entities):
    if (entities[:3] == [0, 0, 0]):
        return "MISC"
    if entities[1:] == [0,0,0]:
        return "PER"
    if entities[:2] == [0,0] and entities[3] == 0:
        return "LOC"
    if entities[2:4] == [0,0] and entities[0] == 0:
        return "ORG"
    return entities
        
        
def connect3(arr, text):
    if len(arr) == 0:
        return []
    arr2 = []
    prev = arr[0]['word']
    prev_end = arr[0]['end']
    prev_start = arr[0]['start']
    #per, org, loc, misc = [0, 0, 0, 0]
    entities = [0, 0, 0, 0]
    whatEntityIsThis(arr[0]['entity'], entities)
    
    for i in range(1, len(arr)):
        if prev_end  == arr[i]['start']:
            prev += arr[i]['word']
            whatEntityIsThis(arr[i]['entity'], entities)

        elif prev_end  == arr[i]['start'] - 1:
            prev += text[arr[i]['start'] - 1] + arr[i]['word']
            whatEntityIsThis(arr[i]['entity'], entities)

            
        else:
            whatEntityIsThis(arr[i]['entity'], entities)
            phrase = prev.replace('#', '')
            #if "ЛЭТИ" not in phrase and "СПбГЭТУ" not in phrase:
            arr2.append({'word': phrase, 'start': prev_start, 'end': prev_end, 'entity': entities})
            prev = arr[i]['word']
            prev_start = arr[i]['start']
            entities = [0, 0, 0, 0]

        prev_end = arr[i]['end']

    phrase = prev.replace('#', '')
    #if "ЛЭТИ" not in phrase and "СПбГЭТУ" not in phrase:
    whatEntityIsThis(arr[len(arr)-1]['entity'], entities)
    arr2.append({'word': phrase, 'start': prev_start, 'end': prev_end, 'entity': entities})
    return arr2

In [4]:
res3 = []

for i in data[:200]:
    if i['object_type'] == 'rpd':
        ner = nlp(i['text'])
        d = {'text': i['text'], 'ner': ner, 'words': connect3(ner, i['text'])}
        res3.append(d)

    


In [5]:
for i in res3:
    #print(i['ner'])
    q = [j['word'] for j in i['words']]
    #print(q)
    print(i['words'])

[]
[]
[{'word': 'Ляпунова', 'start': 100, 'end': 108, 'entity': [5, 0, 0, 0]}, {'word': 'Ляпунова', 'start': 124, 'end': 132, 'entity': [4, 0, 0, 0]}, {'word': 'Ляпунова', 'start': 956, 'end': 964, 'entity': [4, 0, 0, 0]}, {'word': 'Ляпунов', 'start': 1370, 'end': 1377, 'entity': [4, 0, 0, 0]}, {'word': 'Михайлова', 'start': 1450, 'end': 1459, 'entity': [2, 0, 0, 0]}, {'word': 'Ляпунова', 'start': 1565, 'end': 1573, 'entity': [4, 0, 0, 0]}]
[{'word': 'MATLAB', 'start': 838, 'end': 844, 'entity': [1, 0, 0, 4]}, {'word': 'Нелдера-Мида', 'start': 1203, 'end': 1215, 'entity': [5, 0, 0, 0]}, {'word': 'Коши', 'start': 1513, 'end': 1517, 'entity': [3, 0, 0, 0]}]
[{'word': 'Д', 'start': 1, 'end': 2, 'entity': [0, 0, 0, 2]}, {'word': 'Д', 'start': 309, 'end': 310, 'entity': [0, 0, 0, 1]}]
[{'word': 'MATLAB', 'start': 838, 'end': 844, 'entity': [1, 0, 0, 4]}, {'word': 'Нелдера-Мида', 'start': 1203, 'end': 1215, 'entity': [5, 0, 0, 0]}, {'word': 'Коши', 'start': 1513, 'end': 1517, 'entity': [3, 0

In [6]:
symbols = [' ', '.', ',', ':', ';', '(', ')', '!', '?', '"']
def to_full(arr):
    for i in arr:
        for j in i['words']:
            while i['text'][j['start']-1] not in symbols:
                j['start'] -= 1
                if (j['start'] < 0):
                    break
                j['word'] = i['text'][j['start']] + j['word']
            txtlen = len(i['text'])
            while i['text'][j['end']] not in symbols and txtlen - 1 > j['end']:
                j['word'] += i['text'][j['end']]
                j['end'] += 1


In [7]:
to_full(res3)

for i in res3:
    #print(i['words'])
    q = [j['word'] for j in i['words']]
    print(q)

[]
[]
['Ляпунова', 'Ляпунова', 'Ляпунова', 'Ляпунову', 'Михайлова', 'Ляпунова']
['MATLAB', 'Нелдера-Мида', 'Коши']
['Дисциплина', 'Дисциплина']
['MATLAB', 'Нелдера-Мида', 'Коши']
['Производственная']
['компьютером']
[]
['Data Science', 'Data Science', 'CUDA', 'GPU', 'FPGA', 'SoC', 'MATLAB', 'Python', 'R', 'GPU']
[]
['Министерство науки и высшего образования Российской Федерации ФГАОУ ВО «Санкт-Петербургский государственный электротехнический университет «ЛЭТИ» имени В.И.\xa0Ульянова (Ленина']
['СПбГЭТУ', 'ЛЭТИ']
['Robot Operating System', 'ROS']
[]
[]
['Java', 'Java']
['publisher\xadsubscriber', 'RTI Connext DDS', 'OpenSplice', 'ROS', 'ROS2']
[]
['Agile', 'Agile', 'SCRUM', 'User Story', 'Задачи', 'Tasks', 'Спринт', 'Sprint', 'Бэклог', 'Backlog', 'Sprint Burndown Chart', 'SCRUM', 'Product Owner', 'Scrum Master', 'Команда', 'Team', 'Estimate Activity Durations', 'Estimate Activity Resources']
['Семантического Web', 'Семантического Web', 'интеллектуальных', 'интеллектуальных', 'Семантичес

In [8]:
#нумерация, двумерный
def remove_leti(arr):
    new_arr = []
    for i in range(len(arr)):
        new_arr.append([])
        for j in arr[i]['words']:
            if "ЛЭТИ" not in j['word'] and "СПбГЭТУ" not in j['word'] and "науки и высшего образования Российской Федерации" not in j['word']:
                new_arr[i].append({'word': j['word'], 'entity': j['entity']})
    return new_arr

res3_2 = remove_leti(res3)

for i in res3_2:
    #print(i['ner'])
    #print(i)
    q = [j['word'] for j in i]
    print(q)
        
        
#['ЭВМ']
#['Ленина']
#['Ленина', 'М.Ю.\xa0Шестопалов']        

[]
[]
['Ляпунова', 'Ляпунова', 'Ляпунова', 'Ляпунову', 'Михайлова', 'Ляпунова']
['MATLAB', 'Нелдера-Мида', 'Коши']
['Дисциплина', 'Дисциплина']
['MATLAB', 'Нелдера-Мида', 'Коши']
['Производственная']
['компьютером']
[]
['Data Science', 'Data Science', 'CUDA', 'GPU', 'FPGA', 'SoC', 'MATLAB', 'Python', 'R', 'GPU']
[]
[]
[]
['Robot Operating System', 'ROS']
[]
[]
['Java', 'Java']
['publisher\xadsubscriber', 'RTI Connext DDS', 'OpenSplice', 'ROS', 'ROS2']
[]
['Agile', 'Agile', 'SCRUM', 'User Story', 'Задачи', 'Tasks', 'Спринт', 'Sprint', 'Бэклог', 'Backlog', 'Sprint Burndown Chart', 'SCRUM', 'Product Owner', 'Scrum Master', 'Команда', 'Team', 'Estimate Activity Durations', 'Estimate Activity Resources']
['Семантического Web', 'Семантического Web', 'интеллектуальных', 'интеллектуальных', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантическом Web', 'Семантического Web', 'Семантического Web', 'Семантическом Web']
[]
['Logta

In [9]:
for i in range(len(res3_2)):
    #print(i['ner'])
    #print(i)
    res3[i]['words'] = res3_2[i]

In [10]:
# избавиться от повторов
for i in range(len(res3)):
    arr = []
    for j in range(len(res3[i]['words'])):
        if res3[i]['words'][j]['word'] not in str(arr):
            arr.append(res3[i]['words'][j])
        else:
            #добавить entities
            for q in arr:
                if q['word'] == res3[i]['words'][j]['word']:
                    q['entity'] =  [q['entity'][w] + res3[i]['words'][j]['entity'][w] for w in range(4)]



In [11]:
for i in res3:
    for j in i['words']:
        #print(j)
        j['entity'] = choose_one_entity(j['entity'])

for i in range(len(res3)):
    #print(i['words'])
    print(i, [j['word'] for j in res3[i]['words']])
    

0 []
1 []
2 ['Ляпунова', 'Ляпунова', 'Ляпунова', 'Ляпунову', 'Михайлова', 'Ляпунова']
3 ['MATLAB', 'Нелдера-Мида', 'Коши']
4 ['Дисциплина', 'Дисциплина']
5 ['MATLAB', 'Нелдера-Мида', 'Коши']
6 ['Производственная']
7 ['компьютером']
8 []
9 ['Data Science', 'Data Science', 'CUDA', 'GPU', 'FPGA', 'SoC', 'MATLAB', 'Python', 'R', 'GPU']
10 []
11 []
12 []
13 ['Robot Operating System', 'ROS']
14 []
15 []
16 ['Java', 'Java']
17 ['publisher\xadsubscriber', 'RTI Connext DDS', 'OpenSplice', 'ROS', 'ROS2']
18 []
19 ['Agile', 'Agile', 'SCRUM', 'User Story', 'Задачи', 'Tasks', 'Спринт', 'Sprint', 'Бэклог', 'Backlog', 'Sprint Burndown Chart', 'SCRUM', 'Product Owner', 'Scrum Master', 'Команда', 'Team', 'Estimate Activity Durations', 'Estimate Activity Resources']
20 ['Семантического Web', 'Семантического Web', 'интеллектуальных', 'интеллектуальных', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантического Web', 'Семантическом Web', 'Семантического Web', 

In [12]:

#with open('withEntities.json', 'w') as outfile:
#    outfile.write(json.dumps(str(res3), ensure_ascii=False, indent=4))
    